# Decision Tree

In [2]:
# Load our libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Use a nicer style for plots
plt.style.use("seaborn-v0_8-muted")

# Import the regression tree from scikit-learn and a plotting helper
from sklearn.tree import DecisionTreeRegressor, plot_tree
# Import our train_test_split helper
from sklearn.model_selection import train_test_split

## Prepare Dataset into target and features and split them into test train Validation

In [3]:
target_data = pd.read_csv("/Users/rubenstark/Documents/GitHub/Its-Wekk/4 - Data/2 - Ruben/Final_Target_Data_Combined_resid_Trend")

#features Dataset muss noch angepasst werden
features_data = pd.read_csv("/Users/rubenstark/Documents/GitHub/Its-Wekk/4 - Data/2 - Ruben/Working_DataFrame.csv")

In [5]:
# Konvertiere die Datumsspalte in einen datetime-Index (falls nicht bereits)
target_data['Datum'] = pd.to_datetime(target_data['Datum'])
features_data['Datum'] = pd.to_datetime(features_data['Datum'])

# Definiere das Cut-Off-Datum
cutoff_date = pd.Timestamp('2024-10-20 21:00:00+00:00')

# Filtere das Dataset auf Einträge bis einschließlich des Cut-Off-Datums
target_data_cutted = target_data[target_data['Datum'] <= cutoff_date]
features_data_cutted = features_data[features_data['Datum'] <= cutoff_date]

In [6]:
# Split our data intro features and targets
# Teile das Dataset in Features und Zielvariable
y = target_data_cutted["PM10_Combined_Trend_Residual"]  # Zielvariable
X = features_data_cutted.drop(columns=["Datum"])  # Alle Spalten außer der Zielvariable

X.head(10)

# Daten splitten
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=72)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, random_state=72)  # 10% von Gesamt

print("Train Size:", len(X_train))
print("Validation Size:", len(X_val))
print("Test Size:", len(X_test))

Train Size: 17201
Validation Size: 2458
Test Size: 4915


## Decision Tree Regressor

In [7]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)

# Expanding Cross-Validation (5 Splits)
tscv = TimeSeriesSplit(n_splits=5)
results = []

# Use the initial 50% of the data for training (optional, but keeps your requirement in mind)
train_size = int(0.5 * len(X_train))
X_train_init = X_train.iloc[:train_size]
y_train_init = y_train.iloc[:train_size]

# Perform expanding window cross-validation
for fold, (train_index, val_index) in enumerate(tscv.split(X_train_init)):
    # Expanding training data with each fold
    X_train_fold = X_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    X_val_fold = X_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Train the model
    model = DecisionTreeRegressor()
    model.fit(X_train_fold, y_train_fold)
    
    # Predict and calculate MSE
    y_pred = model.predict(X_val_fold)
    mse = mean_squared_error(y_val_fold, y_pred)
    results.append(mse)
    print(f"Fold {fold + 1}: MSE = {mse}")

# Average MSE across all folds
average_mse = np.mean(results)
print(f"Durchschnittlicher MSE über alle Folds: {average_mse}")

Fold 1: MSE = 64.60048714505179
Fold 2: MSE = 65.26163477989127
Fold 3: MSE = 62.40212093212197
Fold 4: MSE = 61.04229798932156
Fold 5: MSE = 58.06193083868737
Durchschnittlicher MSE über alle Folds: 62.27369433701479


## Use ccp_alpha (and other parameters) to optimize the Decision Tree for example when it comes to overfitting

[`DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

In [8]:
np.random.seed(42)

# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
results_ccp = []

# Use the initial 50% of the data for training (optional, but keeps your requirement in mind)
train_size = int(0.5 * len(X_train))
X_train_init = X_train.iloc[:train_size]
y_train_init = y_train.iloc[:train_size]

# Perform expanding window cross-validation
for fold, (train_index, val_index) in enumerate(tscv.split(X_train_init)):
    # Expanding training data with each fold
    X_train_fold = X_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    X_val_fold = X_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Modell trainieren
    tree_ccp = DecisionTreeRegressor(ccp_alpha=0.01)
    tree_ccp.fit(X_train_fold, y_train_fold)
    
    # Vorhersagen machen
    y_pred_ccp = tree_ccp.predict(X_val_fold)
    
    # Berechne den Fehler
    mse_ccp = mean_squared_error(y_val_fold, y_pred_ccp)
    results_ccp.append(mse_ccp)
    
    print(f"Fold {fold + 1}: MSE = {mse_ccp}")

# Durchschnittlichen Fehler über alle Folds berechnen
average_mse_ccp = np.mean(results_ccp)
print(f"Durchschnittlicher MSE über alle Folds: {average_mse_ccp}")


Fold 1: MSE = 62.862562216827534
Fold 2: MSE = 63.09566186655212
Fold 3: MSE = 58.091970327110744
Fold 4: MSE = 55.76480915646729
Fold 5: MSE = 52.66354122276783
Durchschnittlicher MSE über alle Folds: 58.49570895794511


## Crossvalidation to find best alpha 

 Geht nicht mit so vielen Features

In [9]:
# Import our tools for model selection
from sklearn.model_selection import cross_validate, KFold

In [22]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeRegressor
import numpy as np

np.random.seed(42)

# Definiere die Werte für ccp_alpha (Cost Complexity Pruning)
alphas = np.logspace(-4, 0, 50)  # Werte zwischen 10^-4 und 10^0

# Initialisiere Cross-Validation
cv = KFold(n_splits=5, shuffle=True, random_state=72)

# Speicher für Ergebnisse
scores = []  # Durchschnittliche MSE für jedes ccp_alpha
scores_std = []  # Standardabweichung der Scores für Stabilitätsanalyse

# Cross-Validation für jedes ccp_alpha
for alpha in alphas:
    # Decision Tree mit aktuellem ccp_alpha-Wert
    tree_cv = DecisionTreeRegressor(ccp_alpha=alpha, random_state=72)
    
    # Negative MSE, da cross_val_score maximiert; wir wollen minimieren
    mse_scores = cross_val_score(tree_cv, X_train, y_train, cv=cv, scoring="neg_mean_squared_error")
    
    # Durchschnittlichen MSE speichern (negativ, daher multiplizieren mit -1)
    scores.append(-mse_scores.mean())
    scores_std.append(mse_scores.std())

# Optimiere ccp_alpha: Der Wert mit dem niedrigsten MSE
optimal_alpha = alphas[np.argmin(scores)]

print(f"Optimales ccp_alpha: {optimal_alpha}")

Optimales ccp_alpha: 0.12648552168552957


In [23]:
np.random.seed(42)

# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
results_ccp = []

# Use the initial 50% of the data for training (optional, but keeps your requirement in mind)
train_size = int(0.5 * len(X_train))
X_train_init = X_train.iloc[:train_size]
y_train_init = y_train.iloc[:train_size]

# Perform expanding window cross-validation
for fold, (train_index, val_index) in enumerate(tscv.split(X_train_init)):
    # Expanding training data with each fold
    X_train_fold = X_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    X_val_fold = X_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Modell trainieren
    tree_ccp = DecisionTreeRegressor(ccp_alpha=optimal_alpha)
    tree_ccp.fit(X_train_fold, y_train_fold)
    
    # Vorhersagen machen
    y_pred_ccp = tree_ccp.predict(X_val_fold)
    
    # Berechne den Fehler
    mse_ccp = mean_squared_error(y_val_fold, y_pred_ccp)
    results_ccp.append(mse_ccp)
    
    print(f"Fold {fold + 1}: MSE = {mse_ccp}")

# Durchschnittlichen Fehler über alle Folds berechnen
average_mse_ccp = np.mean(results_ccp)
print(f"Durchschnittlicher MSE über alle Folds: {average_mse_ccp}")


Fold 1: MSE = 50.822333454900175
Fold 2: MSE = 45.7341532068147
Fold 3: MSE = 41.7296481074143
Fold 4: MSE = 45.14719606581096
Fold 5: MSE = 35.26947132806262
Durchschnittlicher MSE über alle Folds: 43.74056043260055


## Bagging

Bagging (Bootstrap Aggregating) ist eine Technik, um die Stabilität und Genauigkeit von Machine-Learning-Algorithmen zu verbessern, insbesondere bei Modellen wie Entscheidungsbäumen, die anfällig für hohe Varianz sind. Es basiert auf dem Bootstrapping-Prinzip, bei dem mehrere Trainingssets durch Zufallsstichproben mit Zurücklegen erzeugt werden.

Jeder Baum wird auf einem dieser zufälligen Datensets trainiert, und die Vorhersagen der B Modelle werden durch Mittelung kombiniert. Mathematisch reduziert Bagging die Varianz der Modelle, weil unabhängige Fehler über die Modelle hinweg geglättet werden. So wird die Vorhersage insgesamt stabiler und robuster gegen Variationen in den Trainingsdaten.

Das Ziel ist, Vorhersagefehler durch Mittelung der Outputs der individuellen Modelle zu minimieren, was insgesamt zu einer besseren Modellleistung führt.

In [20]:
# Import the regression tree from scikit-learn and a plotting helper
from sklearn.tree import DecisionTreeRegressor, plot_tree
# Import our train_test_split helper
from sklearn.model_selection import train_test_split
# Import the mean_squared_error function under the alias mse
from sklearn.metrics import mean_squared_error as mse
# Import the resampling helper
from sklearn.utils import resample
# Import the sklearn implementation of bagging
from sklearn.ensemble import BaggingRegressor

np.random.seed(42)



# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
results_bag = []

# Use the initial 50% of the data for training (optional, but keeps your requirement in mind)
train_size = int(0.5 * len(X_train))
X_train_init = X_train.iloc[:train_size]
y_train_init = y_train.iloc[:train_size]

# Perform expanding window cross-validation
for fold, (train_index, val_index) in enumerate(tscv.split(X_train_init)):
    # Expanding training data with each fold
    X_train_fold = X_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    X_val_fold = X_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Modell trainieren
    bagged_trees.fit(X_train_fold, y_train_fold)
    
    # Vorhersagen machen
    y_pred = bagged_trees.predict(X_val_fold)
    
    # Berechne den Fehler
    mse = mean_squared_error(y_val_fold, y_pred)
    results_bag.append(mse)
    
    print(f"Fold {fold + 1}: MSE = {mse}")

# Durchschnittlichen Fehler über alle Folds berechnen
average_mse_bag = np.mean(results_bag)
print(f"Durchschnittlicher MSE über alle Folds: {average_mse_bag}")


Fold 1: MSE = 32.639549650969144
Fold 2: MSE = 32.434337443314206
Fold 3: MSE = 30.033947585850633
Fold 4: MSE = 32.28576945563013
Fold 5: MSE = 27.007602962664453
Durchschnittlicher MSE über alle Folds: 30.88024141968571


## Random Forrest

Random Forests erweitern Bagging, indem sie jedem Baum eine zusätzliche Zufallskomponente hinzufügen. Jeder Baum wird mit einem bootstrap-Sample der Trainingsdaten trainiert, wobei nur ein zufälliger Teil der Features für die Konstruktion des Baums verwendet wird. Dadurch unterscheidet sich Random Forests von klassischem Bagging, bei dem alle Features verfügbar sind.

Die zufällige Auswahl der Features reduziert die Korrelation zwischen den Bäumen und verbessert die Generalisierung des Modells. Üblicherweise wird die Anzahl der verwendeten Features  m  so gewählt, dass  m \approx \sqrt{p} , wobei  p  die Gesamtzahl der Features ist. Wenn  m = p  gesetzt wird, ist Random Forest gleichbedeutend mit einem Bagging-Modell.

In [19]:
# Import the random forest regressor
from sklearn.ensemble import RandomForestRegressor

np.random.seed(42)

# Initialize the random forest regressor
rf = RandomForestRegressor(n_estimators=100, max_features="sqrt")

# Fit the model on the training data
rf.fit(X_train, y_train)

# Compute the predictions on the training and test sets
y_pred_train_rf = rf.predict(X_train)
y_pred_test_rf = rf.predict(X_test)

# Print the mean squared error for training and test sets
print("Train MSE: ", mean_squared_error(y_train, y_pred_train_rf))
print("Test MSE : ", mean_squared_error(y_test, y_pred_test_rf))

# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
results_rf = []

# Use the initial 50% of the data for training (optional, but keeps your requirement in mind)
train_size = int(0.5 * len(X_train))
X_train_init = X_train.iloc[:train_size]
y_train_init = y_train.iloc[:train_size]

# Perform expanding window cross-validation
for fold, (train_index, val_index) in enumerate(tscv.split(X_train_init)):
    X_train_fold, X_val_fold = X_train_init.iloc[train_index], X_train_init.iloc[val_index]
    y_train_fold, y_val_fold = y_train_init.iloc[train_index], y_train_init.iloc[val_index]
    
    # Modell trainieren
    rf.fit(X_train_fold, y_train_fold)
    
    # Vorhersagen machen
    y_pred = rf.predict(X_val_fold)
    
    # Berechne den Fehler
    fold_mse = mean_squared_error(y_val_fold, y_pred)
    results_rf.append(fold_mse)
    
    print(f"Fold {fold + 1}: MSE = {fold_mse}")

# Durchschnittlichen Fehler über alle Folds berechnen
average_mse_rf = np.mean(results_rf)
print(f"Durchschnittlicher MSE über alle Folds: {average_mse_rf}")

Train MSE:  4.5487846956140965
Test MSE :  46.219017242935855
Fold 1: MSE = 33.253726118331834
Fold 2: MSE = 34.105384380851774
Fold 3: MSE = 32.560239049993484
Fold 4: MSE = 33.29749370319174
Fold 5: MSE = 29.66712537085887
Durchschnittlicher MSE über alle Folds: 32.57679372464554


Der Unterschied zwischen der Verwendung eines Integers oder eines Floats bei der Angabe von max_features in einem Random Forest Modell (wie in Scikit-learn) liegt in der Bedeutung des Parameters und wie die Anzahl der maximal zu betrachtenden Features berechnet wird:

1. Wenn max_features ein Integer ist:

	•	Der Wert gibt die exakte Anzahl der maximal zu betrachtenden Features an, die bei der Teilung eines Knotens in jedem Decision Tree berücksichtigt werden sollen.
	•	Beispiel: max_features=3 bedeutet, dass 3 Features aus dem gesamten Feature-Set zufällig ausgewählt werden, um die beste Teilung zu bestimmen.

2. Wenn max_features ein Float ist:

	•	Der Wert gibt einen Prozentsatz der verfügbaren Features an, die verwendet werden sollen. Der Float-Wert muss zwischen 0.0 und 1.0 liegen.
	•	Beispiel: max_features=0.5 bedeutet, dass 50 % der Features (aufgerundet) zufällig ausgewählt werden, um die beste Teilung zu bestimmen.
    

# Lagged Variable mit 1h lag

In [11]:
# Ensure 'Datum' column is in datetime format
target_data_cutted['Datum'] = pd.to_datetime(target_data_cutted['Datum']).copy()

# Create a copy of target_data to apply the offset
lagged_target_variable_1h = target_data.copy()


# Offset von -1 Stunde anwenden
lagged_target_variable_1h['Datum'] = target_data['Datum'] + pd.Timedelta(hours=-1)

lagged_target_variable_1h.head()


/var/folders/6l/nfq1629j4jg8fynztn5kxlph0000gn/T/ipykernel_3969/378574758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data_cutted['Datum'] = pd.to_datetime(target_data_cutted['Datum']).copy()


,Datum,PM10_Combined_Trend_Residual
0,2021-12-31 23:00:00+00:00,75.197962
1,2022-01-01 00:00:00+00:00,51.472071
2,2022-01-01 01:00:00+00:00,32.710483
3,2022-01-01 02:00:00+00:00,24.801767
4,2022-01-01 03:00:00+00:00,9.683660


In [13]:
# Ensure 'Datum' column in features_data is in datetime format
features_data_cutted['Datum'] = pd.to_datetime(features_data['Datum'])

# Merge the dataframes
features_data_lagged = pd.merge(features_data_cutted, lagged_target_variable_1h, on="Datum", how="left")

features_data_lagged.head(5)

/var/folders/6l/nfq1629j4jg8fynztn5kxlph0000gn/T/ipykernel_3969/4226013587.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_data_cutted['Datum'] = pd.to_datetime(features_data['Datum'])


,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch,PM10_Combined_Trend_Residual
0,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.151270,...,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494,51.472071
1,2022-01-01 01:00:00+00:00,-0.489808,-0.193898,-0.043944,-0.525891,-0.191296,-0.000173,-0.000190,-0.000462,-0.000203,...,0.745474,-0.744920,-0.955306,-0.781539,-1.055084,-1.048762,-0.773270,-1.072471,0.357518,32.710483
2,2022-01-01 02:00:00+00:00,-0.279837,-0.000213,-0.011259,-0.058515,-0.000054,-0.000173,-0.122840,-0.000462,-0.000203,...,0.575505,-0.744920,-0.937353,-0.571421,-1.055084,-1.057758,-0.416962,-1.072471,0.385414,24.801767
3,2022-01-01 03:00:00+00:00,-0.209846,-0.000213,-0.011259,-0.350625,-0.000054,-0.000173,-0.000190,-0.000462,-0.000203,...,0.269563,-0.744920,-1.045068,-0.571421,-1.055084,-1.201702,-0.416962,-1.072471,0.617131,9.683660
4,2022-01-01 04:00:00+00:00,-0.139856,-0.000213,0.021425,0.116751,-0.000054,-0.000173,0.245112,-0.000462,0.150864,...,0.575505,-0.331206,-1.063021,-0.781539,-1.055084,-1.255681,-0.654501,-1.072471,1.109860,5.787813


# Models mit Lagged Value trainieren

In [14]:
# Split our data intro features and targets
# Teile das Dataset in Features und Zielvariable
y = target_data_cutted["PM10_Combined_Trend_Residual"]  # Zielvariable
L = features_data_lagged.drop(columns=["Datum"])  # Alle Spalten außer der Zielvariable

X.head(10)

# Split into training and test sets
#L_train, L_test, y_train, y_test = train_test_split(L, y, random_state=72)

# Daten splitten
L_train_val, L_test, y_train_val, y_test = train_test_split(L, y, test_size=0.2, random_state=72)
L_train, L_val, y_train, y_val = train_test_split(L_train_val, y_train_val, test_size=0.125, random_state=72)  # 10% von Gesamt

print("Train Size:", len(L_train))
print("Validation Size:", len(L_val))
print("Test Size:", len(L_test))

Train Size: 17201
Validation Size: 2458
Test Size: 4915


## Normaler Decision Tree

In [15]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)

# Expanding Cross-Validation (5 Splits)
tscv = TimeSeriesSplit(n_splits=5)
L_results = []

# Use the initial 50% of the data for training (optional, but keeps your requirement in mind)
train_size = int(0.5 * len(X_train))
L_train_init = L_train.iloc[:train_size]
y_train_init = y_train.iloc[:train_size]

# Perform expanding window cross-validation
for fold, (train_index, val_index) in enumerate(tscv.split(L_train_init)):
    # Expanding training data with each fold
    L_train_fold = L_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    L_val_fold = L_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Train the model
    L_model = DecisionTreeRegressor()
    L_model.fit(L_train_fold, y_train_fold)
    
    # Predict and calculate MSE
    L_y_pred = L_model.predict(L_val_fold)
    L_mse = mean_squared_error(y_val_fold, L_y_pred)
    L_results.append(L_mse)
    print(f"Fold {fold + 1}: MSE = {L_mse}")

# Average MSE across all folds
L_average_mse = np.mean(L_results)
print(f"Durchschnittlicher MSE über alle Folds: {L_average_mse}")

Fold 1: MSE = 13.387821781376811
Fold 2: MSE = 16.937367877250146
Fold 3: MSE = 23.018912481672988
Fold 4: MSE = 14.04939511204494
Fold 5: MSE = 15.565574523235547
Durchschnittlicher MSE über alle Folds: 16.59181435511609


## Decision Tree mit Optimierung ccp_alpha

In [16]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import numpy as np

np.random.seed(42)

# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
L_results_ccp = []

for fold, (train_index, val_index) in enumerate(tscv.split(L_train_init)):
    # Expanding training data with each fold
    L_train_fold = L_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    L_val_fold = L_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Train the model
    L_tree_ccp = DecisionTreeRegressor(ccp_alpha=0.01)
    L_tree_ccp.fit(L_train_fold, y_train_fold)
    
    # Vorhersagen machen
    L_y_pred_ccp = L_tree_ccp.predict(L_test)
    
    # Berechne den Fehler
    L_mse_ccp = mean_squared_error(y_test, L_y_pred_ccp)
    L_results_ccp.append(L_mse_ccp)
    
    print(f"Fold {fold + 1}: MSE = {L_mse_ccp}")

# Durchschnittlichen Fehler über alle Folds berechnen
L_average_mse_ccp = np.mean(L_results_ccp)
print(f"Durchschnittlicher MSE über alle Folds: {L_average_mse_ccp}")

Fold 1: MSE = 32.9075696981686
Fold 2: MSE = 29.3856920562105
Fold 3: MSE = 33.030461518117406
Fold 4: MSE = 27.75426362417417
Fold 5: MSE = 29.58519425968313
Durchschnittlicher MSE über alle Folds: 30.53263623127076


## Bagging

In [17]:
# Import the regression tree from scikit-learn and a plotting helper
from sklearn.tree import DecisionTreeRegressor, plot_tree
# Import our train_test_split helper
from sklearn.model_selection import train_test_split
# Import the mean_squared_error function under the alias mse
from sklearn.metrics import mean_squared_error as mse
# Import the resampling helper
from sklearn.utils import resample
# Import the sklearn implementation of bagging
from sklearn.ensemble import BaggingRegressor

np.random.seed(42)

B = 100


# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
L_results_bag = []


for fold, (train_index, val_index) in enumerate(tscv.split(L_train_init)):
    # Expanding training data with each fold
    L_train_fold = L_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    L_val_fold = L_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Train the model
    L_bagged_trees = BaggingRegressor(DecisionTreeRegressor(), n_estimators=B, random_state=42)
    L_bagged_trees.fit(L_train_fold, y_train_fold)
    
    # Vorhersagen machen
    L_y_pred = L_bagged_trees.predict(L_test)
    
    # Berechne den Fehler
    L_mse = mean_squared_error(y_test, L_y_pred)
    L_results_bag.append(L_mse)
    
    print(f"Fold {fold + 1}: MSE = {L_mse}")

# Durchschnittlichen Fehler über alle Folds berechnen
L_average_mse_bag = np.mean(L_results_bag)
print(f"Durchschnittlicher MSE über alle Folds: {L_average_mse_bag}")

Fold 1: MSE = 25.426687405799537
Fold 2: MSE = 25.4067706467569
Fold 3: MSE = 25.721519663026307
Fold 4: MSE = 25.293885556058076
Fold 5: MSE = 25.428788985724687
Durchschnittlicher MSE über alle Folds: 25.455530451473102


## Random Forrest

In [18]:
# Import the random forest regressor
from sklearn.ensemble import RandomForestRegressor

# Expanding Cross Validation mit 5 Folds
tscv = TimeSeriesSplit(n_splits=5)

# Liste zur Speicherung der Ergebnisse
L_results_rf = []

for fold, (train_index, val_index) in enumerate(tscv.split(L_train_init)):
    # Expanding training data with each fold
    L_train_fold = L_train_init.iloc[train_index]
    y_train_fold = y_train_init.iloc[train_index]
    
    # Validation data stays fixed (next chunk after the training data)
    L_val_fold = L_train_init.iloc[val_index]
    y_val_fold = y_train_init.iloc[val_index]
    
    # Train the model
    L_rf = RandomForestRegressor(n_estimators=B, max_features="sqrt", random_state=42)
    L_rf.fit(L_train_fold, y_train_fold)

    # Vorhersagen machen
    L_y_pred = L_rf.predict(L_test)
    
    # Berechne den Fehler
    L_fold_mse = mean_squared_error(y_test, L_y_pred)
    L_results_rf.append(L_fold_mse)
    
    print(f"Fold {fold + 1}: MSE = {L_fold_mse}")

# Durchschnittlichen Fehler über alle Folds berechnen
L_average_mse_rf = np.mean(L_results_rf)
print(f"Durchschnittlicher MSE über alle Folds: {L_average_mse_rf}")

Fold 1: MSE = 35.85010727457068
Fold 2: MSE = 33.78032525280543
Fold 3: MSE = 33.1605382971988
Fold 4: MSE = 31.815268076929904
Fold 5: MSE = 31.208938472994195
Durchschnittlicher MSE über alle Folds: 33.1630354748998
